### Итоговый просмотр результатов по 1 этапу и базовому RAG

In [2]:
import pandas as pd
import numpy as np

# Массив с путями к файлам
file_paths = [
    '../eval_base_rag/mean_values.txt',
    '../eval_rag_step_1/step_1_1/mean_values.txt',
    '../eval_rag_step_1/step_1_2/mean_values.txt',
    '../eval_rag_step_1/step_1_3/mean_values.txt',
    '../eval_rag_step_1/step_1_3_1/mean_values.txt',
    '../eval_rag_step_1/step_1_3_1 _(1000 token)/mean_values.txt'
]

# Массив с названиями методов (или столбцов)
method_names = ['Base rag', 'RAG standart_split 1800', 'RAG standart_split 1200/100', 'My html_split 100-220 token', 'html_split 1800 / 100', 'html_split 1000 / 100']

# Инициализация пустого списка для хранения данных
data = []

# Массив для хранения всех уникальных метрик
all_metrics = set()

# Обработка каждого файла из массива путей
for method_index, file_path in enumerate(file_paths):
    method_data = {}
    with open(file_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.split(':')
                metric_name = key.strip()
                metric_value = float(value.strip())
                method_data[metric_name] = metric_value
                all_metrics.add(metric_name)
    
    # Добавление метода в данные
    data.append(pd.Series(method_data, name=method_names[method_index]))

# Создание DataFrame из собранных данных
df = pd.concat(data, axis=1)

# Убедимся, что все метрики присутствуют в каждом столбце
for metric in all_metrics:
    if metric not in df.index:
        df.loc[metric] = np.nan  # Используем NaN для отсутствующих значений

# Заполнение отсутствующих значений NaN (замена '-' на NaN)
df = df.apply(pd.to_numeric, errors='coerce')

# Функция для выделения максимального значения в каждой строке
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: #d3d3d3' if v else '' for v in is_max]

# Применение стиля к DataFrame
styled_df = df.style.apply(highlight_max, axis=1)

# Сохранение DataFrame в CSV файл
df.to_csv('metrics_step_1.csv')

In [3]:
df

,Base rag,RAG standart_split 1800,RAG standart_split 1200/100,My html_split 100-220 token,html_split 1800 / 100,html_split 1000 / 100
faithfulness,0.839286,0.779718,0.850758,0.931481,0.869792,0.923077
answer_relevancy,0.711729,0.675531,0.633100,0.579447,0.500734,0.605244
context_precision,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
context_relevancy,0.754918,0.793711,0.801757,0.774458,0.811815,0.792418
answer_similarity,0.923925,0.912350,0.914586,0.909015,0.908620,0.922121
bleu_score,0.093522,0.075047,0.080700,0.096359,0.097607,0.121988
sim-spacy,0.857789,0.817984,0.807411,0.829006,0.837442,0.844741
cos-sim-TF-IDF,0.265224,0.226048,0.252057,0.256571,0.241461,0.277836
cos-sim-BertModel,0.982229,0.970452,0.968580,0.975761,0.977289,0.981286
human,NaN,0.517241,NaN,NaN,NaN,NaN


In [2]:
styled_df

,Base rag,RAG standart_split 1800,RAG standart_split 1200/100,My html_split 100-220 token,html_split 1800 / 100,html_split 1000 / 100
faithfulness,0.839286,0.779718,0.850758,0.931481,0.869792,0.923077
answer_relevancy,0.711729,0.675531,0.633100,0.579447,0.500734,0.605244
context_precision,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
context_relevancy,0.754918,0.793711,0.801757,0.774458,0.811815,0.792418
answer_similarity,0.923925,0.912350,0.914586,0.909015,0.908620,0.922121
bleu_score,0.093522,0.075047,0.080700,0.096359,0.097607,0.121988
sim-spacy,0.857789,0.817984,0.807411,0.829006,0.837442,0.844741
cos-sim-TF-IDF,0.265224,0.226048,0.252057,0.256571,0.241461,0.277836
cos-sim-BertModel,0.982229,0.970452,0.968580,0.975761,0.977289,0.981286
human,nan,0.517241,nan,nan,nan,nan


'RAG my smart 100-220 token' и 'RAG smart 1000 / 100' имеют примерно одинаковое разбиение по кускам

### RAGAS

**Faithfulness**
- Насколько фактически точен сгенерированный ответ.
- Это измеряет фактическую согласованность сгенерированного ответа с данным контекстом. Он рассчитывается на основе ответа и полученного контекста. Ответ масштабируется до (0,1) диапазона. Чем выше, тем лучше.
Сгенерированный ответ считается верным, если все утверждения, сделанные в ответе, могут быть выведены из данного контекста. Чтобы рассчитать это, сначала определяется набор утверждений из сгенерированного ответа. Затем каждое из этих утверждений проверяется с учетом данного контекста, чтобы определить, можно ли вывести его из данного контекста или нет. Оценка верности дается по делению на
  
**Answer Relevancy**
- Насколько релевантен сгенерированный ответ на вопрос (насколько ответы соответствуют заданным вопросам).
  
**Context Relevancy**
- Метрика оценивает релевантность извлеченного контекста на основе вопроса и контекста.
  
**Answer Semantic Similarity**
- Метрика оценивает семантическое сходство между сгенерированным ответом и эталонным ответом.

### Другие метрики

**BLEU Score**
- BLEU (Bilingual Evaluation Understudy) — это метрика, используемая для оценки качества машинного перевода или сгенерированного текста. Она сравнивает машинно-сгенерированный текст с одним или несколькими эталонными текстами.
  
**Sim-SpaCy**
- Эта метрика оценивает сходство между сгенерированным ответом и эталонным ответом с использованием модели SpaCy. Она измеряет семантическое сходство текстов на основе их векторных представлений.
  
**Cos-Sim-TF-IDF**
- Метрика cosine similarity с использованием TF-IDF оценивает сходство между двумя текстами, измеряя угол между их векторными представлениями в пространстве признаков.
  
 **Cos-Sim-BertModel**
- Эта метрика оценивает семантическое сходство между сгенерированным ответом и эталонным ответом, используя предварительно обученную модель BERT. Она основывается на вычислении косинусного сходства между векторными представлениями текстов, полученными из BERT.
  
 **Human**
- Метрика считается вручную, где 1 - это идеальное совпадение ответа и правильного ответа, 0 - нет свопадения, 0.5 - частичное совпадение. И в конце суммируется и делится на общее количество.